On this homework, you will see the need for *pivoting* (or row swaps) in row reduction algorithms. Two significant (and related) issues arise when row reducing. The first two questions explore these issues. For both of these questions, work without any row swaps.

> ## Make a copy of this notebook (File menu -> Make a Copy...)

### Homework Question 1

Consider the matrix $$\begin{bmatrix} 1 & 2 & 3 & -2\\ 2 & 4 & 1 &  0\\ 3 & 3 & 2 & 5 \\ -1 & 6 & 2 & 1\end{bmatrix}.$$ Row-reduce this matrix by hand, then using your `rowred()` code from lab. What happens? Explain why the issue occurs.

$\begin{bmatrix} 1 & 2 & 3 & -2\\ 2 & 4 & 1 &  0\\ 3 & 3 & 2 & 5 \\ -1 & 6 & 2 & 1\end{bmatrix}$
$\rightarrow$
$\begin{bmatrix} 1 & 2 & 3 & -2\\ 0 & 0 & -5 &  4\\ 0 & -3 & -7 & 11 \\ 0 & 8 & 5 & -1\end{bmatrix}$
$\rightarrow$
$\begin{bmatrix} 1 & 2 & 3 & -2\\ 0 & -3 & -7 & 11 \\ 0 & 0 & -5 &  4\\ 0 & 8 & 5 & -1\end{bmatrix}$
$\rightarrow$
$\begin{bmatrix} 1 & 2 & 3 & -2\\ 0 & -3 & -7 & 11 \\ 0 & 0 & -5 &  4\\ 0 & 0 & -41/3 & 85/3\end{bmatrix}$
$\rightarrow$
$\begin{bmatrix} 1 & 2 & 3 & -2\\ 0 & -3 & -7 & 11 \\ 0 & 0 & -5 &  4\\ 0 & 0 & 0 & 88/5\end{bmatrix}$


In [1]:
import numpy as np

#function to swap two rows given matrix and indices 
def swaprows(A,i,j): 
    A[[i,j]] = A[[j,i]] #swap rows 
    return A

#function to mutliply row of matrix A by scalar 
def rowmult(A,i,c):
    A[i] = c * A[i] #scalar multiplication 
    return A

#function to add mutliple (c) of row i to row j of matrix A
def rowaddmult(A,i,j,c):
    A[j] += c * A[i] #perform row replacement
    return A

# reduces a n array A by iterating through each row
def rowred(A):
    r,c = A.shape
    for x in range(r):
        # simplify each pivot to one
        if A[x,x]!=1:
            rowmult(A, x, 1/A[x,x])
        # add current row times the value of the pivot column of other rows to other rows 
        for y in range(x+1, r):
            rowaddmult(A, x, y, -A[y,x])
            
A = np.array([[1,2,3,-2],[2,4,1,0],[3,3,2,5],[-1,6,2,1]]).astype('float')
rowred(A)
print("The program crashes because of division by zero.")
print(A)

The program crashes because of division by zero.
[[  1.   2.   3.  -2.]
 [ nan  nan -inf  inf]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in double_scalars
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.


### Homework Question 2

Consider the following system of three simultaneous equations in three variables:

$$\begin{align*} 0.0001x_1 &+ 10,000x_3 &&= 10,000.0001 \\ 10,000x_1 &+ 0.0001x_2 &&= 10,000.0001\\10,000x_2 &+ x_3 &&= 10,001\end{align*}$$

1. Write this system as a matrix equation $Ax=v$. 
1. Row-reduce the system and solve it by hand. You should get a nice and simple answer. It may help to express everything as powers of 10, and to note that $$v=\begin{bmatrix}10^4+10^{-4}\\ 10^{4}+10^{-4}\\ 10^4+1\end{bmatrix}.$$ You might even be able to just spot the answer by looking at that, but row-reduce anyway. We'll need the hand-done row-reduction later in this homework.
1. Use your work from Question 11 in the lab to solve the system using NumPy. Do you get the right answer?

In [5]:
def backsub(U, v):
    r,c = U.shape
    for x in reversed(range(0,r)):
        # use the dot product method of back substitution
        v[x] -= v[x+1:r]@U[x,x+1:c]
        # make value equal to one
        v[x] /= U[x,x]
        
A = np.array([[.0001,0,10000,10000.0001],[10000,.0001,0,10000.0001],[0,10000,1,10001]])

rowred(A)
# set v equal to the last col of the augmented matrix
v = A[:,-1].copy()
backsub(A[:,0:-1], v)

print("Answer should be: [1,1,1]")
print("The computer's answer is :", v)

Answer should be: [1,1,1]
The computer's answer is : [1. 2. 1.]


#### What's Going On?

The issue in the first question above is relatively simple: a zero appears in a pivot position. We cannot divide by zero, and so we're stuck.

The second question is more subtle. What happens there is an example of *floating point error*. While a full discussion of floating point error is beyond the scope of these labs, but the following question gives some insight:

### Homework Question 3

Look back at your hand-computed row-reduction from Homework Question 2. For each entry in the row-reduced augemented matrix, write down how many *significant figures* you'd need to write out each number in full. For example, the number 101 requires three significant figures, whereas 100 requires only one, since it can be written as $1e+02$ (i.e. $1\times 10^2$). Likewise, 100.001 ($1.00001e+02$) requires six, as does 0.000100001 ($1.00001e-4$).

Very briefly, floating point numbers can only hold a certain number of significant figures. Numbers requiring more than the limit are rounded. Run the following code, and use the output to find the maximum number of significant figures floats in Python can represent accurately. Explain your answer.
```python
for i in range(20):
    print(i,float(10**i+1)-10**i)
```

Lastly, let's look at where the incorrectly represented numbers came from. Go back to your row reduction, and find the exact places where Python could no longer represent numbers accurately. Explain the following sentence:
> *When we add two numbers of very different magnitudes, we may create numbers that cannot be accurately represented as floats.*

As you saw in Homework Question 2, it is possible to construct relatively simple examples where the limit is exceeded, resulting in very incorrect results that do not round correctly.

Note that we get a floating point issue within *A* itself, even without its augmented column. This will be important in the next lab.

**Our answers:**
<br/>
<br/>
$\begin{bmatrix}
1&1&1\\
1&1&1\\
1&1&21\\
\end{bmatrix}$
$\begin{bmatrix}
9\\
17\\
21\\
\end{bmatrix}$

In python numbers are represented with a fixed number of bits, these bits can only represent a range of decimal integers. If a number has more significant digits than can be represented by that number of bits (64 in python) the least significant digits are truncated. 

In [45]:
for i in range(20):
    print(i,float(10**i+1)-10**i)
# Python only keeps 17 significant digits. The code above should return 1 for all i, however due to Python sig-figs
# the lsd is cut off when there are more than 17 digits. 

# Specifically, python uses the IEEE-754 standard which specifies 53 bits of precision in a double. 
# From: https://docs.python.org/3/tutorial/floatingpoint.html


0 1.0
1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 0.0
17 0.0
18 0.0
19 0.0


To solve (or at least reduce) the problem we saw above, we use a strategy called *Maximal Partial Pivoting*. The idea is this: In a given row, look at all numbers in the column *below* the pivot. If there is a number whose magnitude (absolute value) is larger than the pivot, swap that row with the current one (if two rows have the same magnitude in that column, just pick the first to swap with). Then proceed with regular row reduction.

### Homework Question 4

By hand, carry out row reduction with MPP for the matrix $$\begin{bmatrix} 1 & 2 & 2 \\ 2 & 1 & 2 \\ 2 & -1 & 2\end{bmatrix}$$

$\begin{bmatrix}
1&2&2\\
2&1&2\\
2&-1&2\\
\end{bmatrix}$
$\Rightarrow$
$\begin{bmatrix}
2&1&2\\
1&2&2\\
2&-1&2\\
\end{bmatrix}$
$\Rightarrow$
$\begin{bmatrix}
2&1&2\\
0&1.5&1\\
0&-2&0\\
\end{bmatrix}$
$\Rightarrow$
$\begin{bmatrix}
2&1&2\\
0&-2&0\\
0&1.5&1\\
\end{bmatrix}$
$\Rightarrow$
$\begin{bmatrix}
2&1&2\\
0&-2&0\\
0&0&1\\
\end{bmatrix}$
$\Rightarrow$
$\begin{bmatrix}
1&.5&1\\
0&1&0\\
0&0&1\\
\end{bmatrix}$




### Homework Question 5

To find the index of the largest entry of a vector, use the command `np.argmax(v)`. Use this to modify your `rowred(A)` routine to create a new routine `rowredpivot(A)` that implements MPP. Test your code on the matrix from the last question, as well as the matrix from Homework Question 1.

**Note: When testing for a swap, be sure to only test entries below the current pivot. You will need to be a little careful with the output from `np.argmax()`.**

In [10]:
def rowredpivot(A):
    r,c = A.shape
    for x in range(r):
        # for all rows not at the bottom
        if x < r-1:
            # find row index with the highest value in the specified column
            pivot = np.argmax(np.absolute(A[:,x]));
            # convert the pivot (which could be an array) into an int
            if type(pivot)==type(A):
                pivot = pivot[0]
            # set the next pivot row to the row defined by pivot
            swaprows(A,x,pivot)
            
        if A[x,x]!=1:
            rowmult(A, x, 1/A[x,x])
        for y in range(x+1, r):
            rowaddmult(A, x, y, -A[y,x])

A = np.array([[1,2,2],[2,1,2],[2,-1,2]]).astype('float')
rowredpivot(A)
print("Solved array from question 5:\n", A, end="\n\n")

            
Q1 = np.array([[1,2,3,-2],[2,4,1,0],[3,3,2,5],[-1,6,2,1]]).astype('float')
rowredpivot(Q1)
print("Solved array from question 1:\n", Q1)

Solved array from question 5:
 [[ 1.   0.5  1. ]
 [-0.   1.  -0. ]
 [ 0.   0.   1. ]]

Solved array from question 1:
 [[ 1.          1.          0.66666667  1.66666667]
 [ 0.          1.          0.38095238  0.38095238]
 [ 0.          0.          1.         -2.07317073]
 [-0.         -0.         -0.          1.        ]]


### Homework Question 6

Repeat Homework Question 2 above with your new routine. You should get the right answer this time. Lastly, carry out row-reduction with MPP by hand for this system. You should still find that there are places where we get rounding errors. Can you explain why the answer you get from your routine is nonetheless correct? We will explore this more in depth on the next homework.

In [6]:
A = np.array([[.0001,0,10000,10000.0001],[10000,.0001,0,10000.0001],[0,10000,1,10001]])

rowredpivot(A)
v = A[:,-1].copy()
backsub(A[:,0:-1], v)

print("Answer should be: [1,1,1]")
print(v)

Answer should be: [1,1,1]
[1. 1. 1.]
